<a href="https://colab.research.google.com/github/Servat0r/HLT-Project-2023/blob/master/IT5_base_full_training_over_SquaD_IT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Test Settings:

1. **Model**: IT5-base;
2. **Dataset**: Squad-IT;
3. **Examples**: whole dataset;
4. **Other**: can compare to same training size with `MT5-Base` over both standard LMQG Squad and its augmented version.

### 1. Preliminaries

#### Mounting and Installing

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/MyDrive/Colab Notebooks"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks


In [ ]:
%run utils.ipynb

In [ ]:
NUM_EPOCHS = 5

#### Imports

In [ ]:
from transformers import AutoModel, AutoTokenizer, AutoModelForSeq2SeqLM, AdamW, DataCollatorWithPadding
from datasets import load_dataset, Dataset, load_from_disk, load_metric
import numpy as np
import evaluate
import torch
import os

In [ ]:
model_checkpoint='gsarti/it5-base'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
tokenizer.add_special_tokens({'sep_token': '<sep>', 'additional_special_tokens': ['<answer>', '<context>']})

3

### 2. Dataset Loading and Preprocessing

We will use [`squad_it`](https://huggingface.co/datasets/squad_it), a variant of the `SquaD` dataset adapted to `Question Generation` tasks.

In [ ]:
def load_and_preprocess_squad_it_dataset(
    dataset_name='squad_it', train_dataset_name='squad_it_qg_train',
    eval_dataset_name='squad_it_qg_eval', test_dataset_name='squad_it_qg_test',
    shuffle_seed=None, train_select=None, eval_select=None, train_dataset_split=0.8,
    use_extra_ids=False,
):
  dataset_loading_result = get_dataset(dataset_name, train_dataset_name, eval_dataset_name, test_dataset_name)
  local = dataset_loading_result['local']
  if local:
    train_dataset = dataset_loading_result['train']
    validation_dataset = dataset_loading_result['eval']
    test_dataset = dataset_loading_result['test']
  if not local:
    datasets = load_dataset(dataset_name)
    dev_dataset = datasets['train'].remove_columns(['id'])
    test_dataset = datasets['test'].remove_columns(['id'])

    new_test_dataset = {'answer': [], 'question': [], 'context': []}
    test_dataset.map(lambda sample: build_answers_squad_it(sample, new_test_dataset))
    test_dataset = Dataset.from_dict(new_test_dataset)

    new_dev_dataset = {'answer': [], 'question': [], 'context': []}
    dev_dataset.map(lambda sample: build_answers_squad_it(sample, new_dev_dataset))
    del dev_dataset
    dev_dataset = Dataset.from_dict(new_dev_dataset)

    train_dataset_length = int(train_dataset_split * dev_dataset_length) + 1 if 0 <= train_dataset_split <= 1 else int(train_dataset_split)

    train_dataset = dev_dataset.shuffle(seed=shuffle_seed).select(range(train_dataset_length))
    validation_dataset = dev_dataset.shuffle(seed=shuffle_seed).select(range(train_dataset_length, dev_dataset_length))
    print(f"Train dataset has {len(train_dataset)} items. Validation dataset has {len(validation_dataset)} items.")

    train_dataset.save_to_disk(train_dataset_name)
    validation_dataset.save_to_disk(eval_dataset_name)
    test_dataset.save_to_disk(test_dataset_name)

  if train_select:
    train_dataset = train_dataset.shuffle(seed=shuffle_seed).select(range(train_select))
  if eval_select:
    validation_dataset = validation_dataset.shuffle(seed=shuffle_seed).select(range(eval_select))

  build_train_feature = lambda sample: build_train_feature_squad_it(sample, use_extra_ids=use_extra_ids)
  train_dataset = train_dataset.map(build_train_feature).remove_columns(['answer', 'context'])
  validation_dataset = validation_dataset.map(build_train_feature).remove_columns(['answer', 'context'])
  test_dataset = test_dataset.map(build_train_feature).remove_columns(['answer', 'context'])

  tokenized_train_dataset = train_dataset.map(tokenizer_function, batched=True).remove_columns(['answer_context', 'question'])
  tokenized_validation_dataset = validation_dataset.map(tokenizer_function, batched=True).remove_columns(['answer_context', 'question'])
  tokenized_test_dataset = test_dataset.map(lambda samples: tokenizer_function(samples, input_ids_padding="max_length", train_dataset=train_dataset), batched=True).remove_columns(['answer_context'])

  tokenized_train_dataset.set_format("torch")
  tokenized_validation_dataset.set_format("torch")
  tokenized_test_dataset.set_format("torch")

  return (train_dataset, validation_dataset, test_dataset), (tokenized_train_dataset, tokenized_validation_dataset, tokenized_test_dataset)

In [ ]:
(train_dataset, validation_dataset, test_dataset), (tokenized_train_dataset, tokenized_validation_dataset, tokenized_test_dataset) = \
  load_and_preprocess_squad_it_dataset(shuffle_seed=42, train_select=None, eval_select=None, train_dataset_split=0.8, use_extra_ids=True)

### 3. Model Loading and configuration

#### Loading

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [ ]:
model.resize_token_embeddings(len(tokenizer))

You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embedding dimension will be 32106. This might induce some performance reduction as *Tensor Cores* will not be available. For more details about this, or help on choosing the correct value for resizing, refer to this guide: https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#requirements-tc


Embedding(32106, 768)

#### Configuration

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

device(type='cuda')

In [ ]:
optimizer, train_dataloader, eval_dataloader, lr_scheduler, \
  num_training_steps = get_training_configuration(train_batch_size=8, eval_batch_size=8, tokenizer=tokenizer, learning_rate=1e-4, num_epochs=NUM_EPOCHS)

27080


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
train_dataloader

In [ ]:
for batch in train_dataloader:
  batch['input_ids'].to(device)
  break

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


### 4. Fine-tuning

#### Execution

In [ ]:
training_results_dict = main_training_loop(
    model, device, optimizer, train_dataloader, eval_dataloader,
    lr_scheduler, num_training_steps, num_epochs=NUM_EPOCHS, metrics=None,
    eval_strategy='epoch', eval_every=2000, model_save_path='squad_it_reduced_it5base_full_training_test',
    early_stopping=False, early_stopping_min_delta=1e-3, early_stopping_patience=5,
    tokenizer=tokenizer, num_beams=4, tokenize_predictions_output=False, start_epoch=0
  )

epoch_train_losses = training_results_dict['epoch_train_losses']
epoch_eval_losses = training_results_dict['epoch_eval_losses']
epoch_eval_metrics = training_results_dict['epoch_eval_metrics']
epoch = training_results_dict['epoch']
print(epoch_train_losses, epoch_eval_losses, epoch_eval_metrics, sep='\n')

  0%|          | 0/27080 [00:00<?, ?it/s]

  0%|          | 0/6770 [00:00<?, ?it/s]

<ipython-input-3-fe8eb841f70d>:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels_batch = torch.tensor(batch['labels'])


Epoch 0: Train Loss = 1.4088494777679443, Eval Loss = 1.5851269960403442
Save this model (y/n)?> y
Continue training (y/n)?> y


<ipython-input-3-fe8eb841f70d>:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels_batch = torch.tensor(batch['labels'])


Epoch 1: Train Loss = 1.3430821895599365, Eval Loss = 1.5353460311889648
Save this model (y/n)?> y
Continue training (y/n)?> n
[1.4088494777679443, 1.3430821895599365]
[1.5851269960403442, 1.5353460311889648]
[]


In [ ]:
model.save_pretrained(f'squad_it_reduced_it5base_full_training_example_(epoch {epoch})')

### 5. Analysis of the Results

### Calculating BLEU and ROUGE score

In [ ]:
loss_tracker=[]
metrics_tracker=[]
metrics = {
    'bleu': get_bleu_config(tokenizer),
    'nist': get_nist_config(tokenizer),
    'rouge': get_rouge_config(tokenizer),
}

In [ ]:
P = True

In [ ]:
if P:
  model = AutoModelForSeq2SeqLM.from_pretrained('squad_it_reduced_it5base_full_training_test_epoch1', local_files_only=True)

In [ ]:
if P:
  optimizer, train_dataloader, eval_dataloader, lr_scheduler, \
    num_training_steps = get_training_configuration(train_batch_size=4, eval_batch_size=4, tokenizer=tokenizer, learning_rate=1e-3, num_epochs=2)

21664


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
if P:
  device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
  model.to(device)
  print()

In [ ]:
tokenized_test_dataset = tokenized_test_dataset.remove_columns(['question'])

In [ ]:
from tqdm.auto import tqdm
test_dataloader = DataLoader(tokenized_test_dataset, shuffle=True, batch_size=8, collate_fn=DataCollatorWithPadding(tokenizer=tokenizer))
test_loss_tracker, test_metrics_tracker, num_test_steps = [], [], len(test_dataloader)
test_progress_bar = tqdm(range(num_test_steps))
test_loss = evaluation_loop(
    model, device, optimizer, test_dataloader, lr_scheduler, test_loss_tracker, test_metrics_tracker, metrics, test_progress_bar,
    num_beams=4, tokenizer=tokenizer, tokenize_predictions_output=False,
)

  0%|          | 0/1279 [00:00<?, ?it/s]

<ipython-input-3-fe8eb841f70d>:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels_batch = torch.tensor(batch['labels'])
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `None` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:377: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `None` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Metrics = {'bleu': {'bleu': 0.21719074346232226, 'precisions': [0.4929065845309139, 0.2662431576442158, 0.17146896934807856, 0.11620336997657235], 'brevity_penalty': 0.9604604321302002, 'length_ratio': 0.9612219120332751, 'translation_length': 119477, 'reference_length': 124297}, 'nist': {'nist_mt': 5.573417007954352}, 'rouge': {'rouge1': 0.4467628581344718, 'rouge2': 0.26202171634703975, 'rougeL': 0.41612384688497495, 'rougeLsum': 0.41607454595385074}}


In [ ]:
test_dataset[18:30]['question']

['Dal 1947 al 1967, quanto è aumentato il prezzo del petrolio?',
 'Dal 1947 al 1967, quanto è aumentato il prezzo del petrolio?',
 "Quando ha iniziato l' OPEC a riadattare i prezzi del petrolio?",
 "Quando ha iniziato l' OPEC a riadattare i prezzi del petrolio?",
 "Quando l' olio è tornato ai suoi livelli di Bretton Woods?",
 "Quando la Siria e l' Egitto hanno lanciato un attacco a sorpresa contro Israele?",
 "Quante volte di più le altre nazioni hanno dovuto pagare per il petrolio dopo l' attacco a sorpresa?",
 "Quante volte di più le altre nazioni hanno dovuto pagare per il petrolio dopo l' attacco a sorpresa?",
 "Quante volte di più le altre nazioni hanno dovuto pagare per il petrolio dopo l' attacco a sorpresa?",
 "Perchè lo scià d' Iran ha rilasciato un' intervista?",
 "Perchè l' OPEC ha portato il prezzo del petrolio a 5,11 dollari?",
 'Quando hanno portato il prezzo del petrolio a 5,11 dollari?']

In [ ]:
predictions = model.generate(tokenized_test_dataset['input_ids'][18:30].to(device), max_length=64, num_beams=10)

In [ ]:
tokenizer.batch_decode(predictions, skip_special_tokens=True)

['quanto è aumentato il prezzo del petrolio dal 1947 al 1967?',
 'quanto è aumentato il prezzo del petrolio dal 1947 al 1967?',
 "quando l' opec ha iniziato ad adeguare i prezzi per riflettere il deprezzamento?",
 "quando l' opec ha iniziato ad adeguare i prezzi per riflettere il deprezzamento?",
 "in quale anno l' opec ha aumentato i prezzi del petrolio?",
 "in quale data la siria e l' egitto hanno lanciato un attacco a sorpresa su israele?",
 'quale paese era il secondo esportatore mondiale di petrolio?',
 'quante volte il prezzo del petrolio è aumentato?',
 "quanto è giusto che d' ora in poi si paghi di più per il petrolio?",
 'qual è stato il prezzo del petrolio a yom kippur?',
 "perché l' opec ha aumentato il prezzo del petrolio del 70%?",
 "quando l' opec ha aumentato il prezzo del petrolio?"]

In [ ]:
test_loss

1.3398029804229736

In [ ]:
bertscore = compute_bert_score(
    test_dataset, tokenized_test_dataset, model, device, tokenizer, batch_size=16,
    lang='en', model_type=None, max_length=100, num_beams=4, num_candidates=1
)

  0%|          | 0/640 [00:00<?, ?it/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
bertscore

{'precision': (0.8774897929172896, 0.017205166699050246),
 'recall': (0.8753100660812508, 0.01819535053596315),
 'f1': (0.8761216766899451, 0.016528745360921457)}

In [ ]:
np.mean([v[0] for v in bertscore.values()])

0.8763071785628286